# Exam 2
## Jiaqi Ma
## USC ID: 3028009001

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

1.

In [4]:
df = pd.read_csv('Caravan.csv')

In [5]:
df.shape

(5822, 86)

In [6]:
df = df.drop(['PVRAAUT', 'AVRAAUT'], axis = 1)

In [7]:
train = df[1000:]
test = df[:1000]

In [8]:
X_train = train.drop('Purchase', axis = 1)
y_train = train['Purchase']

In [9]:
X_test = test.drop('Purchase', axis = 1)
y_test = test['Purchase']

a) A random forest model with 500 trees, max_features = 29, and random_state = 0.

In [10]:
forest = RandomForestClassifier(n_estimators = 500, 
                                max_features = 29, 
                                random_state = 0)
forest.fit(X_train, y_train)
pred1 = forest.predict(X_test)

In [11]:
cm1 = pd.crosstab(pred1, y_test)
cm1.index.name = 'Prediction'
cm1

Purchase,No,Yes
Prediction,,
No,915,52
Yes,26,7


In [12]:
# test set precision
print('Precision:',
      round((cm1.iloc[1][1]/
            (cm1.iloc[1][0] + cm1.iloc[1][1]))*100, 2),
      '%')

Precision: 21.21 %


b) A boosting model with max_depth = 4. Use 1000 trees, learning rate 0.01, and random_state = 0.

In [13]:
boosting = GradientBoostingClassifier(n_estimators = 1000,
                                      learning_rate = 0.01,
                                      max_depth = 4,
                                      random_state = 0)
boosting.fit(X_train, y_train)
pred2 = boosting.predict(X_test)

In [14]:
cm2 = pd.crosstab(pred2, y_test)
cm2.index.name = 'Prediction'
cm2

Purchase,No,Yes
Prediction,,
No,937,57
Yes,4,2


In [15]:
# test set precision
print('Precision:',
      round((cm2.iloc[1][1]/
            (cm2.iloc[1][0] + cm2.iloc[1][1]))*100, 2),
      '%')

Precision: 33.33 %


c) A multilayer perceptron with three hidden layers having 400 nodes 

In [16]:
# scale the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
mlp = MLPClassifier(solver = 'lbfgs', 
                    random_state = 0,
                    activation = 'tanh', 
                    hidden_layer_sizes = [400, 400, 400],
                    max_iter = 2000)
mlp.fit(X_train_scaled, y_train)
pred3 = mlp.predict(X_test_scaled)

In [18]:
cm3 = pd.crosstab(pred3, y_test)
cm3.index.name = 'Prediction'
cm3

Purchase,No,Yes
Prediction,,
No,885,48
Yes,56,11


In [19]:
# test set precision
print('Precision:',
      round((cm3.iloc[1][1]/
            (cm3.iloc[1][0] + cm3.iloc[1][1]))*100, 2),
      '%')

Precision: 16.42 %


d) Logistic regression with the default value for C.

In [20]:
lg = LogisticRegression(solver = 'lbfgs', max_iter = 900)
lg.fit(X_train_scaled, y_train)
pred4 = lg.predict(X_test_scaled)

In [21]:
cm4 = pd.crosstab(pred4, y_test)
cm4.index.name = 'Prediction'
cm4

Purchase,No,Yes
Prediction,,
No,934,59
Yes,7,0


In [22]:
# test set precision
print('Precision:',
      round((cm4.iloc[1][1]/
            (cm4.iloc[1][0] + cm4.iloc[1][1]))*100, 2),
      '%')

Precision: 0.0 %


e) Repeat part (d) with a cut-off probability value equal to 0.25. LogisticRegression uses 0.50 as cut-off default value. It uses the cut-off value to predict the category. If the probability is larger than 0.5 the model predicts category Yes, otherwise it predicts No. Function model.predict_proba( ) may help to this end.

In [23]:
lg = LogisticRegression(solver = 'lbfgs', max_iter = 900)
lg.fit(X_train_scaled, y_train)
prob5 = lg.predict_proba(X_test_scaled)

In [24]:
pred5 = (prob5[:,1] > 0.25).astype('int')

In [25]:
cm5 = pd.crosstab(pred5, y_test)
cm5.index = ['No', 'Yes']
cm5.index.name = 'Prediction'
cm5

Purchase,No,Yes
Prediction,,
No,917,48
Yes,24,11


In [26]:
# test set precision
print('Precision:',
      round((cm5.iloc[1][1]/
            (cm5.iloc[1][0] + cm5.iloc[1][1]))*100, 2),
      '%')

Precision: 31.43 %


2.

In [27]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person = 60)

In [28]:
print(faces.images.shape, 
      faces.data.shape, 
      faces.target.shape, 
      faces.target_names.shape)

(1348, 62, 47) (1348, 2914) (1348,) (8,)


In [29]:
X = faces.data
y = faces.target

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify = y,
                                                    random_state = 42,
                                                    test_size = 0.25)

a) Fit an SVC model as shown. Find the test accuracy rate.

In [31]:
svc = SVC(kernel = 'rbf', gamma = 0.01)
model = make_pipeline(svc)
model.fit(X_train, y_train)

preda = model.predict(X_test)

In [32]:
# test accuracy rate
print('Accuracy Rate:',
      round(accuracy_score(y_test, preda)*100, 2),
      '%')

Accuracy Rate: 39.47 %


b) To improve the model accuracy we will transform the features using Principal Components, a topic we review in ISE 535. Use first 150 principal components as the new features of the model, as shown below, then report the test accuracy rate.

In [33]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 150, 
          svd_solver = 'randomized', 
          whiten = True, 
          random_state = 42)
svc = SVC(kernel = 'rbf', gamma = 0.01)
model = make_pipeline(pca, svc)
model.fit(X_train, y_train)

predb = model.predict(X_test)

In [34]:
# test accuracy rate
print('Accuracy Rate:',
      round(accuracy_score(y_test, predb)*100, 2),
      '%')

Accuracy Rate: 69.73 %


c) Add argument class_weight='balanced' to the SVC( ) line above, run the code again and find the accuracy rate. Here we modify the loss function by weighting the loss of each row category by its class weight.

In [35]:
pca = PCA(n_components = 150, 
          svd_solver = 'randomized', 
          whiten = True, 
          random_state = 42)
svc = SVC(kernel = 'rbf', 
          gamma = 0.01, 
          class_weight = 'balanced')
model = make_pipeline(pca, svc)
model.fit(X_train, y_train)

predc = model.predict(X_test)

# test accuracy rate
print('Accuracy Rate:',
      round(accuracy_score(y_test, predc)*100, 2),
      '%')

Accuracy Rate: 74.48 %


d) Use GridSearchCV to search best values for C and gamma. Try the grid values shown below. Report the test accuracy rate.

In [36]:
pca = PCA(n_components=150, 
          svd_solver = 'randomized', 
          whiten = True, 
          random_state = 42)
svc = SVC(kernel = 'rbf', class_weight = 'balanced')
model = make_pipeline(pca, svc)
param_grid = {'svc__C': [1, 5, 10],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, cv = 5)
grid.fit(X_train, y_train)
model = grid.best_estimator_

predd = model.predict(X_test)

# test accuracy rate
print('Accuracy Rate:',
      round(accuracy_score(y_test, predd)*100, 2),
      '%')

Accuracy Rate: 84.87 %


In [37]:
# best values for C and gamma
grid.best_params_

{'svc__C': 5, 'svc__gamma': 0.005}

e) Report the confusion matrix (for test set) of the best model in a dataframe with rows
identifying the true name of the person and columns identifying the predicted name. What person
has the largest error rate (that is, the person that is most difficult to predict)?

In [38]:
# confusion matrix
cm = confusion_matrix(y_test, predd)
cm = pd.DataFrame(cm, index = faces.target_names, 
                  columns = faces.target_names)
cm.index.name = 'True Name'
cm.columns.name = 'Predicted Name'
cm

Predicted Name,Ariel Sharon,Colin Powell,Donald Rumsfeld,George W Bush,Gerhard Schroeder,Hugo Chavez,Junichiro Koizumi,Tony Blair
True Name,,,,,,,,
Ariel Sharon,10,4,0,5,0,0,0,0
Colin Powell,1,53,0,4,0,0,0,1
Donald Rumsfeld,0,2,23,4,0,1,0,0
George W Bush,0,5,0,128,0,0,0,0
Gerhard Schroeder,0,0,0,2,24,0,0,1
Hugo Chavez,0,3,0,3,1,10,0,1
Junichiro Koizumi,0,0,1,2,0,0,10,2
Tony Blair,0,0,1,6,1,0,0,28


In [39]:
error_rate = []
for i in range(len(cm)):
    error_rate.append(1-cm.iloc[i][i]/sum(cm.iloc[i]))

error_rate = pd.DataFrame(error_rate, columns = ['Error_Rate'])
error_rate

,Error_Rate
0,0.473684
1,0.101695
2,0.233333
3,0.037594
4,0.111111
5,0.444444
6,0.333333
7,0.222222


In [40]:
# the person that is most difficult to predict is Ariel Sharon
faces.target_names[error_rate.idxmax(axis = 0)]

array(['Ariel Sharon'], dtype='<U17')

In [41]:
print('The largest error rate is:',
      round(error_rate['Error_Rate'].max()*100, 2),
      '%')

print('The accuracy Rate is:',
      round(accuracy_score(y_test, predd)*100, 2),
      '%')

The largest error rate is: 47.37 %
The accuracy Rate is: 84.87 %
